# DOLPHOT/WFPC2

Notebook to do photometry with DOLPHOT and WFPC2 data. It calls the bash routines and the outpout redirected to a file so the notebook doesn't get too long

In [1]:
import glob
import os
import subprocess
from shutil import copy2
from dolphot import DolphotParameters

In [2]:
listainitials = glob.glob('Originals/u971*c[0-1]m.fits')
#drzinitial = glob.glob('Originals/u971010*drz.fits')
#drzinitialfile =drzinitial[0]
#drzinitialfilename = drzinitialfile.split('.')[0].split('/')[1]
#listainitials.append(drzinitial[0])
#print(listainitials)
finaldrizorfile = 'Driz/final_drz_sci.fits'
listainitials.append(finaldrizorfile)
drzfilename = finaldrizorfile.split('.')[0].split('/')[1]
print(drzfilename)
#print(listainitials)
datadir = 'Try/'

for i in listainitials:
    copy2(i,datadir)

final_drz_sci


In [3]:
pattern = 'u*c0m.fits'
listacom = glob.glob('{}{}'.format(datadir,pattern))
listafits = glob.glob('{}{}'.format(datadir,'*fits'))

# 3.2 wfpc2mask

In [4]:
for i in listacom:
    t = i.replace('c0m','c1m')
    commandmask = 'wfpc2mask {} {} &>> wfpc2mask.log'.format(i,t)
    #print(commandmask)
    ! {commandmask}
! head wfpc2mask.log

Reading FITS file Try/u971210fm_c0m.fits
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading FITS file Try/u971210fm_c1m.fits
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800


# 3.3 splitgroups: Split UVIS Chips

In [5]:
dirfits = datadir+'*.fits'
#print(dirfits)
commandsplit = 'splitgroups {} &>> splitgroups.log'.format(dirfits)
#print(commandsplit)
! {commandsplit}
! head splitgroups.log

Reading FITS file Try/final_drz_sci.fits: 1549x1527
Reading BINTABLE extension: 2307x504
Writing FITS file Try/final_drz_sci.chip0.fits: 1549x1527
Writing FITS file Try/final_drz_sci.chip1.fits: 2307x504
Reading FITS file Try/u9710103m_c0m.fits
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Reading IMAGE extension: 800x800
Writing FITS file Try/u9710103m_c0m.chip1.fits: 800x800


# 3.4 calcsky: Calculate Sky for Image

In [6]:
rin=10
rout = 25
step = 2
sigmalow = 2.25
sigmahigh = 2.00
listacomall = glob.glob('{}{}'.format(datadir,'u971*m_c0m.*.fits'))
listachipdrz = glob.glob('{}{}'.format(datadir,drzfilename+'*chip*'))
listacomall.extend(listachipdrz)
#listafits = glob.glob('{}{}'.format(datadir,'*fits'))
#print(listachipdrz)
for i in listacomall:
    name = i.split('.fits')[0]
    #print(name)
    #name = 'Try/u9710103m_c0m.chip1'
    calcskycommad = 'calcsky {} {} {} {} {} {} &>> calcsky.log'.format(name,rin,rout,step,sigmalow, sigmahigh)
    #print(calcskycommad)
    #! {calcskycommad}

#! head calcsky.log


# Build Param file

This is a great exampleto build a param file in htis git repo:

https://github.com/jonathansick/delphinus

Ill use this to build the params file.


Do it only one or it will add

In [12]:
dolpa = DolphotParameters()



# Try chip3

In [13]:
dolpa.setup_image('Try/final_drz_sci.chip0',ref=True)

In [14]:
#listacomallchip3 = glob.glob('{}{}'.format(datadir,'u971*[!103]m_c0m.chip3.fits'))
listacomallchip = glob.glob('{}{}'.format(datadir,'u971*m_c0m.chip[1-4].fits'))

for i in listacomallchip:
    dolpa.setup_image(i)
    #print(i)

In [15]:
dolpa.write_parameters('dolphot.param')

In [16]:
!head dolphot.param

Nimg = 504
img0_file = Try/final_drz_sci.chip0
img0_psfa = 3.00 0.00 0.00 0.00 0.00 0.00
img0_psfb = 3.00 0.00 0.00 0.00 0.00 0.00
img0_psfc = 0.00 0.00 0.00 0.00 0.00 0.00
img0_aprad = 0.50
img0_apsky = 15.00 25.00
img0_RSky = 15.00 25.00
img0_RAper = 3.00
img0_RChi = 2.00
